In [1]:
import pandas as pd
import numpy as np

## Identify Overlapping Peaks of Two Peak Tables

In [44]:
def ppm_calculation(x, y):
    return abs(x-y)*10e6/y

In [45]:
def n_overlapping_peaks(t_1, t_2, col_name_mz, col_name_rt, ppm_threshold, rt_tolerance):
    # Calculating number of overlapping peaks between two peak tables. Make sure t_2 has less rows than t_1
    n = 0
    for i in range(len(t_2)):
        mz = t_2[col_name_mz].iloc[i]
        rt = t_2[col_name_rt].iloc[i]
        if len(t_1[(ppm_calculation(mz, t_1[col_name_mz])<=ppm_threshold) & (abs(table_1[col_name_rt] - rt) < rt_tolerance)])>0:
            n += 1
    return n

### Local CMD VS HPC CMD

In [61]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_cmd/ms-dial/AlignResult-20231272341.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [62]:
table_2 = pd.read_csv("../manuscript/peak_table/local_cmd/ms-dial/AlignResult-2023129133.txt", delimiter="\t", header=4)

In [63]:
len(table_1)

25951

In [64]:
len(table_2)

25445

In [65]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

18857

### HPC NF VS Local NF

In [51]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_nf/ms-dial/AlignResult-202212142025.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [52]:
table_2 = pd.read_csv("../manuscript/peak_table/local_nf/ms-dial/AlignResult-20231271922.txt", delimiter="\t", header=4)

In [53]:
len(table_1)

25951

In [54]:
len(table_2)

25445

In [55]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

18857

### HPC CMD VS HPC NF

In [56]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_cmd/ms-dial/AlignResult-20231272341.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [57]:
table_2 = pd.read_csv("../manuscript/peak_table/hpc_nf/ms-dial/AlignResult-202212142025.txt", delimiter="\t", header=4)

In [58]:
len(table_1)

25951

In [59]:
len(table_2)

25951

In [60]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

25951

### Local CMD VS Local NF

In [68]:
table_1 = pd.read_csv("../manuscript/peak_table/local_cmd/ms-dial/AlignResult-2023129133.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [69]:
table_2 = pd.read_csv("../manuscript/peak_table/local_nf/ms-dial/AlignResult-20231271922.txt", delimiter="\t", header=4)

In [70]:
len(table_1)

25445

In [71]:
len(table_2)

25445

In [72]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

25445

### Overlapping Between Local and Benchmark

In [103]:
table_1 = pd.read_csv("../manuscript/peak_table/local_nf/ms-dial/AlignResult-20231271922.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [104]:
table_2 = pd.read_excel("../manuscript/benchmark_libs/lib.xlsx", sheet_name="QE HF dataset_TableS8", header=6)

In [105]:
len(table_1)

25445

In [106]:
table_1.head()

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,SA1,SA2,SA3,SA4,SA5,SB1,SB2,SB3,SB4,SB5
0,0,0.562,100.01421,Unknown,[M+2H]2+,NaN,0.3,False,NaN,NaN,...,8714982.00,7387047.5,10247660.0,8517238.0,6663123.0,7660357.00,6522387.5,6514624.0,9974121.0,9562859.0
1,1,35.967,100.01428,Unknown,[M+NH4]2+,adduct linked to 2150_[M+H]+,0.9,False,NaN,NaN,...,765282.90,942749.0,1229463.0,880391.2,1132224.9,907290.06,1239561.4,1271661.1,1253242.1,1348959.9
2,2,0.500,100.01443,Unknown,[M+H]2+,adduct linked to 3419_[M+Na]+,1.0,False,NaN,NaN,...,8001129.00,13925881.0,16331838.0,16200029.0,18298616.0,6891007.50,15492309.0,16130891.0,17702120.0,20029132.0
3,3,0.592,100.01444,Unknown,[M+2H]2+,NaN,0.9,False,NaN,NaN,...,8898810.00,10046386.0,9382828.0,10474542.0,10473924.0,12107133.00,8653832.0,9658213.0,12457828.0,11770822.0
4,4,31.647,100.06959,Unknown,[M+H]+,NaN,1.0,False,NaN,NaN,...,824087.44,1108073.5,1230883.0,1154022.8,1614246.2,2215985.00,2037453.5,2112368.2,2052877.8,1913928.4


In [107]:
table_1["Average Mz"]

0         100.01421
1         100.01428
2         100.01443
3         100.01444
4         100.06959
            ...    
25440    1490.57178
25441    1490.92615
25442    1491.03601
25443    1497.48169
25444    1497.52856
Name: Average Mz, Length: 25445, dtype: float64

In [108]:
table_1["Average Rt(min)"]

0         0.562
1        35.967
2         0.500
3         0.592
4        31.647
          ...  
25440    30.313
25441     4.751
25442     4.750
25443    31.699
25444    30.323
Name: Average Rt(min), Length: 25445, dtype: float64

In [109]:
len(table_2)

836

In [110]:
table_2.head()

,Feature number,Compound formula,Adduct,Average Mz,Average Rt(min),Fold change,p-value,Compound concentration ratio,Unnamed: 8,Unnamed: 9
0,1,C21H26N2S2,+H,371.1617,18.85,0.474166,0.000481,1/2,NaN,NaN
1,2,C25H31NO6,+H,442.2233,17.66,21.230982,0.000001,16/1,NaN,NaN
2,3,C14H17N3O2S,+H,292.1118,3.01,1.905322,0.019356,2/1,NaN,NaN
3,4,C27H31N7OS,+H,502.2395,22.78,1.062849,0.289866,1/1,NaN,NaN
4,5,C20H23NO4,+H,342.1706,3.37,1.066033,0.399052,1/1,NaN,NaN


In [111]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

401

### Overlapping Between HPC and Benchmark

In [112]:
table_1 = pd.read_csv("../manuscript/peak_table/hpc_nf/ms-dial/AlignResult-202212142025.txt", delimiter="\t", header=4)

/Users/xinsongdu/.pyenv/versions/miniconda3-latest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [113]:
table_2 = pd.read_excel("../manuscript/benchmark_libs/lib.xlsx", sheet_name="QE HF dataset_TableS8", header=6)

In [114]:
len(table_1)

25951

In [115]:
n_overlapping_peaks(table_1, table_2, "Average Mz", "Average Rt(min)", 10, 0.3)

390